In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/Shared drives/TecnologieSemantiche')

#Initialization

In [3]:
!pip install ampligraph
!pip install tensorflow==1.13.2

     |████████████████████████████████| 168 kB 5.2 MB/s 
     |████████████████████████████████| 482 kB 44.3 MB/s 
     |████████████████████████████████| 2.7 MB 44.6 MB/s 
     |████████████████████████████████| 6.4 MB 37.5 MB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
     |████████████████████████████████| 120 kB 59.3 MB/s 
     |████████████████████████████████| 561 kB 45.6 MB/s 
     |████████████████████████████████| 42 kB 916 kB/s 
     |████████████████████████████████| 69 kB 7.4 MB/s 
     |████████████████████████████████| 41 kB 535 kB/s 
     |████████████████████████████████| 90 kB 10.3 MB/s 
     |████████████████████████████████| 121 kB 54.9 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 100 kB 9.0 MB/s 
  Created wheel for commonmark: filename=CommonMark-0.5.4-py3-none-any.whl size=30768 sha256=b9275850d8934b2fa83492a44edf24d5351c2a1b84dfd1f33c58406c1c9632d4
  Stored in directory: /root/.cache/pip/whe

     |████████████████████████████████| 92.7 MB 1.3 MB/s 
     |████████████████████████████████| 50 kB 6.9 MB/s 
     |████████████████████████████████| 367 kB 55.3 MB/s 
     |████████████████████████████████| 3.2 MB 43.8 MB/s 
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.2.0
    Uninstalling importlib-metadata-4.2.0:
      Successfully uninstalled importlib-metadata-4.2.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalling tensorflow-2.8.0:
      Successfully uninstalled tensorflow-2.8.0
ERROR: pip's dependenc

In [4]:
import numpy as np
import pandas as pd
import ampligraph
import tensorflow as tf
import requests
from ampligraph.datasets import load_from_csv

ampligraph.__version__

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

'1.4.0'

#EX1: Load Ampligraph Dataset

In [ ]:
'''import requests
from ampligraph.datasets import load_from_csv
url = 'https://ampligraph.s3-eu-west-1.amazonaws.com/datasets/GoT.csv'
open('GoT.csv', 'wb').write(requests.get(url).content) 
X_ = load_from_csv('.', 'GoT.csv', sep=',')
X_'''


#Load Our Dataset

##Generate dbpedia URIs for the dataset 

In [ ]:
##Uncomment next lines if you need
import csv 

df = pd.read_csv('Triple.csv', names=["subject","relation","object"])

dbpedia = "http://dbpedia.org/resource/"

for index,row in df.iterrows():
  s = dbpedia + row['subject'].replace(" ","_")
  p = dbpedia + row['relation'].replace(" ","_")
  o = dbpedia + row['object'].replace(" ","_")
  
  with open('HP_uri.csv', 'a+',newline='') as f:
      writer = csv.writer(f)
      writer.writerow([s,p,o])
      f.close()


In [ ]:
with open('HP_uri_2.csv', 'rb') as f:
  X = load_from_csv('.','HP_uri_2.csv', sep=',')
  #X = X.to_numpy()
X

array([['http://dbpedia.org/resource#subject',
        'http://dbpedia.org/resource#relation',
        'http://dbpedia.org/resource#object'],
       ['http://dbpedia.org/resource#central_character',
        'http://dbpedia.org/resource#is',
        'http://dbpedia.org/resource#Harry_Potter'],
       ['http://dbpedia.org/resource#central_character',
        'http://dbpedia.org/resource#is',
        'http://dbpedia.org/resource#wizard'],
       ...,
       ['http://dbpedia.org/resource#Harry',
        'http://dbpedia.org/resource#awakens_faces',
        'http://dbpedia.org/resource#Voldemort'],
       ['http://dbpedia.org/resource#curse',
        'http://dbpedia.org/resource#rebounds',
        'http://dbpedia.org/resource#Harry'],
       ['http://dbpedia.org/resource#lives',
        'http://dbpedia.org/resource#surviving',
        'http://dbpedia.org/resource#characters_effects_Voldemort']],
      dtype=object)

#Train/Test

In [ ]:
from ampligraph.evaluation import train_test_split_no_unseen 

num_test = int(len(X) * (20 / 100))

data = {}
data['train'], data['test'] = train_test_split_no_unseen(X, test_size=num_test, seed=0, allow_duplication=False) 

print('Train set size: ', data['train'].shape)
print('Test set size: ', data['test'].shape)

Train set size:  (861, 3)
Test set size:  (215, 3)


In [ ]:
from ampligraph.latent_features import HolE

#optimizer
#adam     MRR = 0.44

#loss
#pairwise         MRR = 0.36

#regularizer
#'p':2  MRR = 0.44

#lr
#1e-4   MRR = 0.36
model = HolE(batches_count=100, 
                seed=0, 
                epochs=200, 
                k=150, 
                eta=5,
                optimizer='adam', 
                optimizer_params={'lr':1e-2}, 
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

#Other models

In [ ]:
from ampligraph.latent_features import ComplEx

#MRR = 0.37
model = ComplEx(batches_count=100, 
                seed=0, 
                epochs=200, 
                k=150, 
                eta=5,
                optimizer='adam', 
                optimizer_params={'lr':1e-3}, #va peggio per 1e-2 e 1e-5
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

In [ ]:
from ampligraph.latent_features import TransE

#MRR = 0.21
model = TransE(batches_count=100, 
                seed=0, 
                epochs=200, 
                k=150, 
                eta=5,
                optimizer='adam', 
                optimizer_params={'lr':1e-3}, 
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

In [ ]:
from ampligraph.latent_features import DistMult

#MRR= 0.34
model = DistMult(batches_count=100, 
                seed=0, 
                epochs=200, 
                k=150, 
                eta=5,
                optimizer='adam', 
                optimizer_params={'lr':1e-3}, 
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

#Fit

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)

model.fit(data['train'], early_stopping = False)

Average HolE Loss:   0.275317: 100%|██████████| 200/200 [04:08<00:00,  1.24s/epoch]


#Evaluate Test

In [ ]:
from ampligraph.evaluation import evaluate_performance

filter_triples = np.concatenate((data['train'], data['test']))
ranks = evaluate_performance(data['test'], 
                             model=model, 
                             filter_triples=filter_triples,   # Corruption strategy filter defined above 
                             use_default_protocol=True, # corrupt subj and obj separately while evaluating
                             verbose=True)

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 215/215 [00:01<00:00, 139.79it/s]


In [ ]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

MRR: 0.73
Hits@10: 0.77
Hits@3: 0.75
Hits@1: 0.71


#Evaluate Unseen triples

In [ ]:
#GoT Unseen Triples
X_unseen = np.array([
    ['http://dbpedia.org/resource/Jorah_Mormont','http://dbpedia.org/resource/SPOUSE','http://dbpedia.org/resource/Daenerys_Targaryen'],
    ['http://dbpedia.org/resource/Tyrion_Lannister','http://dbpedia.org/resource/SPOUSE','http://dbpedia.org/resource/Missandei'],
    ["http://dbpedia.org/resource/King's_Landing",'http://dbpedia.org/resource/SEAT_OF','http://dbpedia.org/resource/House_Lannister_of_Casterly_Rock'],
    ['http://dbpedia.org/resource/Sansa_Stark','http://dbpedia.org/resource/SPOUSE','http://dbpedia.org/resource/Petyr_Baelish'],
    ['http://dbpedia.org/resource/Daenerys_Targaryen','http://dbpedia.org/resource/SPOUSE','http://dbpedia.org/resource/Jon_Snow'],
    ['http://dbpedia.org/resource/Daenerys_Targaryen','http://dbpedia.org/resource/SPOUSE','http://dbpedia.org/resource/Craster'],
    ['http://dbpedia.org/resource/House_Stark_of_Winterfell','http://dbpedia.org/resource/IN_REGION','http://dbpedia.org/resource/The_North'],
    ['http://dbpedia.org/resource/House_Stark_of_Winterfell','http://dbpedia.org/resource/IN_REGION','http://dbpedia.org/resource/Dorne'],
    ['http://dbpedia.org/resource/House_Tyrell_of_Highgarden','http://dbpedia.org/resource/IN_REGION','http://dbpedia.org/resource/Beyond_the_Wall'],
    ['http://dbpedia.org/resource/Brandon_Stark','http://dbpedia.org/resource/ALLIED_WITH','http://dbpedia.org/resource/House_Stark_of_Winterfell'],
    ['http://dbpedia.org/resource/Brandon_Stark','http://dbpedia.org/resource/ALLIED_WITH','http://dbpedia.org/resource/House_Lannister_of_Casterly_Rock'],    
    ['http://dbpedia.org/resource/Rhaegar_Targaryen','http://dbpedia.org/resource/PARENT_OF','http://dbpedia.org/resource/Jon_Snow'],
    ['http://dbpedia.org/resource/House_Hutcheson','http://dbpedia.org/resource/SWORN_TO','http://dbpedia.org/resource/House_Tyrell_of_Highgarden'],
    ['http://dbpedia.org/resource/Daenerys_Targaryen','http://dbpedia.org/resource/ALLIED_WITH','http://dbpedia.org/resource/House_Stark_of_Winterfell'],
    ['http://dbpedia.org/resource/Daenerys_Targaryen','http://dbpedia.org/resource/ALLIED_WITH','http://dbpedia.org/resource/House_Lannister_of_Casterly_Rock'],
    ['http://dbpedia.org/resource/Jaime_Lannister','http://dbpedia.org/resource/PARENT_OF','http://dbpedia.org/resource/Myrcella_Baratheon'],
    ['http://dbpedia.org/resource/Robert_I_Baratheon','http://dbpedia.org/resource/PARENT_OF','http://dbpedia.org/resource/Myrcella_Baratheon'],
    ['http://dbpedia.org/resource/Cersei_Lannister','http://dbpedia.org/resource/PARENT_OF','http://dbpedia.org/resource/Myrcella_Baratheon'],
    ['http://dbpedia.org/resource/Cersei_Lannister','http://dbpedia.org/resource/PARENT_OF','http://dbpedia.org/resource/Brandon_Stark'],
    ["http://dbpedia.org/resource/Tywin_Lannister",'http://dbpedia.org/resource/PARENT_OF','http://dbpedia.org/resource/Jaime_Lannister'],
    ["http://dbpedia.org/resource/Missandei",'http://dbpedia.org/resource/SPOUSE','http://dbpedia.org/resource/Grey_Worm'],
    ["http://dbpedia.org/resource/Brienne_of_Tarth",'http://dbpedia.org/resource/SPOUSE','http://dbpedia.org/resource/Jaime_Lannister']
])

In [ ]:
#HP_unseen triples

df = pd.read_csv('Unseen_uri.csv', names=["subject","relation","object"])
X_unseen = df.to_numpy()

In [5]:
#Uncomment next lines in case you want to load trained model
'''from ampligraph.utils import restore_model

df = pd.read_csv('Unseen_uri.csv', names=["subject","relation","object"])
X_unseen = df.to_numpy()
filter_triples = np.load('filter_triples.npy', allow_pickle=True)
model = restore_model(model_name_path = 'HP_Hole')'''

In [6]:
from ampligraph.evaluation import evaluate_performance

unseen_filter = np.array(list({tuple(i) for i in np.vstack((filter_triples, X_unseen))}))

ranks_unseen = evaluate_performance(
    X_unseen, 
    model=model, 
    filter_triples=unseen_filter,   # Corruption strategy filter defined above 
    corrupt_side = 's+o',
    use_default_protocol=False, # corrupt subj and obj separately while evaluating
    verbose=True
)

scores = model.predict(X_unseen)

100%|██████████| 35/35 [00:00<00:00, 77.17it/s]


In [7]:
from scipy.special import expit
probs = expit(scores)

output = pd.DataFrame(list(zip([' '.join(x) for x in X_unseen], 
                      ranks_unseen, 
                      np.squeeze(scores),
                      np.squeeze(probs))), 
             columns=['statement', 'rank', 'score', 'prob']).sort_values("score")

output

,statement,rank,score,prob
30,http://dbpedia.org/resource/Harry http://dbped...,1565,-0.953934,0.278094
20,http://dbpedia.org/resource/Harry http://dbped...,1351,-0.518585,0.373183
29,http://dbpedia.org/resource/Hermione http://db...,1482,-0.376428,0.406989
25,http://dbpedia.org/resource/Tom_Marvolo_Riddle...,1499,-0.250149,0.437787
10,http://dbpedia.org/resource/Ron http://dbpedia...,1254,-0.183073,0.454359
24,http://dbpedia.org/resource/Hermione http://db...,1165,-0.167906,0.458122
32,http://dbpedia.org/resource/Snape http://dbped...,1258,-0.160629,0.459929
21,http://dbpedia.org/resource/Ron http://dbpedia...,1263,-0.125585,0.468645
22,http://dbpedia.org/resource/Ron http://dbpedia...,1219,-0.117500,0.470659
5,http://dbpedia.org/resource/Harry_Potter http:...,1290,-0.088283,0.477944


#Save Evaluated statements in nt format

In [15]:
##Convert triples in nt format
import pandas as pd 

lines = []
th = 1000

for index, row in output.iterrows():
  if row['rank'] < th:
      statement = row['statement']

      namespace = "http://dbpedia.org/resource"

      url_subject,url_predicate,url_object = statement.split(" ")
      
      url_subject = url_subject.replace(" ","_")
      url_predicate = url_predicate.replace(" ","_")
      url_object = url_object.replace(" ","_")

      subject = url_subject.replace("http://dbpedia.org/resource/" , "")
      predicate = url_predicate.replace("http://dbpedia.org/resource/", "")
      objectt = url_object.replace("http://dbpedia.org/resource/", "")

      #Subject type
      lines.append("<" + namespace + "#" + subject + ">" +
                   " <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>" +  
                   " <" + namespace + "#subject> .")
      #Subject name
      lines.append("<" + namespace + "#" + subject + ">" +
                   " <" + namespace + "#name>" +
                   " \"" + subject + "\" .")
      #Subject link
      lines.append("<" + namespace + "#" + subject + ">" +
                   " <" + namespace + "#link>" + 
                   " \"" + url_subject + "\" .")
     
      #Object type
      lines.append("<" + namespace + "#" + objectt + ">" +
                   " <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>" +  
                   " <" + namespace + "#object> .")
      #Object name
      lines.append("<" + namespace + "#" + objectt + ">" +
                   " <" + namespace + "#name>" +
                   " \"" + objectt + "\" .")
      #Object link
      lines.append("<" + namespace + "#" + subject + ">" +
                   " <" + namespace + "#link>" + 
                   " \"" + url_subject + "\" .")
      #Triple
      s = "<" + namespace + "#" + subject + ">"
      p = "<" + url_predicate + ">"
      o = "<" + namespace + "#" + objectt + "> ."
      lines.append(s + " " + p + " " + o)

with open('triples_5.nt', 'w+') as f:
    f.write('\n'.join(lines))